In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd



## Reading the data

In [2]:
data_path = 'clean_conversation.txt'

In [3]:
input_texts = []
target_texts = []
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(600, len(lines) - 1)]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [4]:
len(input_texts)

566

In [5]:
len(target_texts)

566

In [6]:
zippedList =  list(zip(input_texts, target_texts))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output']) 

In [7]:
lines.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


## Preparing input data for the Encoder

In [8]:
input_lines = list()
for line in lines.input:
    input_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines ) 
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines ) 

length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Input max length is {}'.format( max_input_length ))

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
print( 'Number of Input tokens = {}'.format( num_input_tokens))


Input max length is 22
Encoder input data shape -> (566, 22)
Number of Input tokens = 518


## Preparing input data for the Decoder

In [9]:
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines ) 
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines ) 

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))


Output max length is 74
Decoder input data shape -> (566, 74)
Number of Output tokens = 1692


## Preparing target data for the Decoder 

In [10]:
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
decoder_target_data = np.array( onehot_output_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (566, 74, 1692)


## Defining the Model

In [11]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    132608      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    433152      ['input_2[0][0]']                
                                                                                              

## Training

In [12]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=250) 
model.save( 'model.h5' ) 

Epoch 1/250
5/5 [==============================] - 21s 2s/step - loss: 1.4403 - accuracy: 0.0843
Epoch 2/250
5/5 [==============================] - 11s 2s/step - loss: 1.4277 - accuracy: 0.1371
Epoch 3/250
5/5 [==============================] - 14s 3s/step - loss: 1.3273 - accuracy: 0.0998
Epoch 4/250
5/5 [==============================] - 16s 3s/step - loss: 1.1774 - accuracy: 0.0724
Epoch 5/250
5/5 [==============================] - 14s 3s/step - loss: 1.1386 - accuracy: 0.0776
Epoch 6/250
5/5 [==============================] - 14s 3s/step - loss: 1.1215 - accuracy: 0.0769
Epoch 7/250
5/5 [==============================] - 14s 3s/step - loss: 1.1097 - accuracy: 0.0724
Epoch 8/250
5/5 [==============================] - 15s 3s/step - loss: 1.1039 - accuracy: 0.1025
Epoch 9/250
5/5 [==============================] - 14s 3s/step - loss: 1.0988 - accuracy: 0.1392
Epoch 10/250
5/5 [==============================] - 14s 3s/step - loss: 1.0925 - accuracy: 0.1422
Epoch 11/250
5/5 [===========

## Inference models

In [14]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [15]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    print(tokens_list)
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')


In [16]:
enc_model , dec_model = make_inference_models()

enc_model.save( 'enc_model.h5' ) 
dec_model.save( 'dec_model.h5' ) 
model.save( 'model.h5' ) 

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        print(sampled_word_index)
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        print(sampled_word,sampled_word_index)
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()
    


{'end': 1, 'start': 2, 'i': 3, 'a': 4, 'the': 5, 'you': 6, 'of': 7, 'to': 8, 'and': 9, 'is': 10, 'not': 11, 'do': 12, 'that': 13, 'in': 14, 'what': 15, 'am': 16, 'as': 17, 'it': 18, 'have': 19, 'are': 20, 'my': 21, 'get': 22, "i'm": 23, 'when': 24, 'be': 25, 'me': 26, 'can': 27, 'an': 28, 'feel': 29, 'by': 30, 'for': 31, 'or': 32, 'cross': 33, "don't": 34, 'no': 35, 'with': 36, 'about': 37, 'software': 38, 'on': 39, 'all': 40, 'but': 41, 'like': 42, 'very': 43, 'he': 44, 'think': 45, 'how': 46, 'at': 47, 'computer': 48, 'which': 49, 'so': 50, 'one': 51, 'your': 52, 'any': 53, 'from': 54, 'could': 55, 'much': 56, 'was': 57, 'we': 58, 'say': 59, 'human': 60, 'really': 61, 'emotion': 62, 'been': 63, 'just': 64, 'more': 65, 'if': 66, 'why': 67, 'right': 68, "that's": 69, 'said': 70, 'feeling': 71, 'yet': 72, 'than': 73, 'too': 74, 'hard': 75, 'time': 76, 'who': 77, 'know': 78, 'only': 79, 'well': 80, 'would': 81, 'capable': 82, 'should': 83, 'yes': 84, 'programmed': 85, 'myself': 86, 'has'

KeyboardInterrupt: 